In [3]:
#Upload The neede file
from google.colab import files
files.upload()


Saving data_after_mice.csv to data_after_mice.csv


In [4]:
#Install Auto SKLearn
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

for _ in range(3):
    try:
        import autosklearn.classification
        break
    except:
        pass
else:
    raise ImportError("failed to import from autosklearn")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 10 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,072 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fron

In [10]:
#Import Required File Into DataFrame and Compute in missing values
import pandas as pd
import numpy as np
selected_features = ['grade_C','grade_D','grade_E',
                    'grade_F','grade_G','grade_B',
                     'sub_grade_B5','sub_grade_B4',
                     'total_rec_int','sub_grade_E5',
                     'sub_grade_A5','sub_grade_C5',
                     'total_pymnt_inv','int_rate']
df=pd.read_csv("data_after_mice.csv",header=0,encoding='utf-8')

fullData = df[selected_features]

target_col = ["int_rate"]
cat_cols= []
num_cols= list(set(list(fullData.columns))-set(cat_cols)-set(target_col))
num_cat_cols = num_cols+cat_cols
for var in num_cat_cols:
    if fullData[var].isnull().any()==True:
        fullData[var+'_NA']=fullData[var].isnull()*1
#Impute numerical missing values with mean
fullData[num_cols] = fullData[num_cols].fillna(fullData[num_cols].mean())
#Impute categorical missing values with -9999
fullData[cat_cols] = fullData[cat_cols].fillna(value = -9999)
for var in cat_cols:
    number = LabelEncoder()
    fullData[var] = number.fit_transform(fullData[var].astype('str'))

features = list(set(list(fullData.columns))-set(target_col))
print(fullData[features].max())

fullData[features] = fullData[features]/fullData[features].max()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


grade_C                1.00
sub_grade_E5           1.00
sub_grade_B4           1.00
sub_grade_B5           1.00
grade_F                1.00
sub_grade_C5           1.00
grade_D                1.00
total_pymnt_inv    57777.58
sub_grade_A5           1.00
total_rec_int      24205.62
grade_E                1.00
grade_G                1.00
grade_B                1.00
dtype: float64


In [11]:
#Run SKLearn
from sklearn.model_selection import train_test_split
import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(fullData[features], fullData[target_col],test_size=0.20, random_state=42)
automl = autosklearn.regression.AutoSklearnRegressor()
automl.fit(X_train, y_train)

print(automl.show_models())
predictions = automl.predict(X_test)
print("R2 score:", sklearn.metrics.r2_score(y_test, predictions))
automl.fit(X_train, y_train)
y_hat_test = automl.predict(X_test)
x_hat_train=automl.predict(X_train)

/usr/local/lib/python3.6/dist-packages/autosklearn/automl.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-03-22 18:34:26,139:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 18:34:26,164:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 18:34:26,267:AutoMLSMBO(1)::e8124558ef342d1dad262471ed472a28] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/r2_regression_dense
[WARNING] [2019-03-22 18:34:28,171:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 18:34:30,183:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 18:34:32,201:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 18:34:34,218:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models bet

/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-03-22 19:30:05,145:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:07,156:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:09,171:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:11,185:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:13,203:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:15,222:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:17,238:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:30:19,257

/usr/local/lib/python3.6/dist-packages/autosklearn/automl.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-03-22 19:34:20,925:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:34:20,959:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:34:21,074:AutoMLSMBO(1)::e8124558ef342d1dad262471ed472a28] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/r2_regression_dense
[WARNING] [2019-03-22 19:34:22,979:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:34:24,987:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:34:27,004:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models better than random - using Dummy Score!
[WARNING] [2019-03-22 19:34:29,019:EnsembleBuilder(1):e8124558ef342d1dad262471ed472a28] No models bet